Clean up the vehicle idling dataset:
```
!pip install geopandas
!pip install rtree
!pip install pandas fiona shapely pyproj rtree
!apt install libspatialindex-dev
```



In [0]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt

In [0]:
df = pd.read_json('VehicleIdling_org.json')
df = df[['Latitude', 'Longitude']]

In [0]:
gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))

In [4]:
print(gdf.head())

    Latitude  Longitude                                      geometry
0  40.757072 -73.979574  POINT (-73.97957360153642 40.75707175090273)
1  40.595541 -74.164521  POINT (-74.16452110964595 40.59554115156192)
2  40.585445 -73.822096  POINT (-73.82209634713989 40.58544503178869)
3  40.730946 -73.993249  POINT (-73.99324919465766 40.73094608874433)
4  40.675663 -73.927688  POINT (-73.92768817444949 40.67566320795236)


In [0]:
areas = geopandas.read_file('Neighborhood Tabulation Areas.geojson')

In [6]:
areas.columns

Index(['ntacode', 'shape_area', 'county_fips', 'ntaname', 'shape_leng',
       'boro_name', 'boro_code', 'geometry'],
      dtype='object')

In [7]:
print(areas.crs)
gdf.crs = {'init' :'epsg:4326'}
print(gdf.crs)

{'init': 'epsg:4326'}
{'init': 'epsg:4326'}


In [8]:
result = geopandas.sjoin(gdf, areas[['ntaname', 'geometry']], how="inner", op='intersects')
result.head()

,Latitude,Longitude,geometry,index_right,ntaname
0,40.757072,-73.979574,POINT (-73.97957360153642 40.75707175090273),100,Midtown-Midtown South
142,40.763887,-73.980164,POINT (-73.98016353684847 40.7638870522965),100,Midtown-Midtown South
239,40.757546,-73.978740,POINT (-73.97873964579706 40.75754644086999),100,Midtown-Midtown South
377,40.750746,-73.986036,POINT (-73.98603600797342 40.75074607561738),100,Midtown-Midtown South
494,40.752706,-73.990681,POINT (-73.99068077325056 40.75270629442309),100,Midtown-Midtown South


In [9]:
output1 = result[['ntaname', 'index_right']]
output2 = output1.groupby(['ntaname']).count()
output2.rename(columns={'index_right':'count'}, inplace=True)
output2.head()

,count
ntaname,
Airport,2
Allerton-Pelham Gardens,42
Annadale-Huguenot-Prince's Bay-Eltingville,19
Arden Heights,25
Astoria,105


In [0]:
output2.to_json('VehicleIdling.json')